Make sure to pip install pickle, praw, and imdbpy

In [1]:
import json
import requests
import praw
import pprint
from collections import Counter
import pickle



Pickle all data to be more easily reusable

In [2]:
r = praw.Reddit(user_agent='reddit_hype')
deadpool = r.search('deadpool')
subreddits = Counter()
for x in range(10):
	entry = next(deadpool)
	subreddit = entry.subreddit
	subreddits[subreddit] += 1


with open( "pickled/saved_queries.p", "wb" ) as f:
	pickle.dump(subreddits, f)



print('Pickled')

Pickled


Print list of top 100 imdb movies

In [111]:
import tmdbsimple as tmdb
with open( "API_keys/tmdb_api_key.txt", "rb" ) as f:
     tmdb_api_key = f.read()

tmdb.API_KEY = tmdb_api_key
discover = tmdb.Discover()

all_responses = {}
for n in range(2010,2016):
    responses= []
    for x in range(1,6):
        response = discover.movie(page=x, year=n)
        responses.append(response['results'])    

    # flatten list of lists
    responses = [val for sublist in responses for val in sublist]
    all_responses[n] = responses

In [113]:
# get all dicts together
aggregate_responses = [all_responses[year_of_movies] for year_of_movies in all_responses]

# flatten
aggregate_responses = [ movie for yearlist in aggregate_responses for movie in yearlist]
all_title_dates = [(movie['title'], movie['release_date']) for movie in aggregate_responses]
all_title_dates


[('Inception', '2010-07-14'),
 ('Terminator Salvation', '2009-05-20'),
 ('Avatar', '2009-12-10'),
 ('Black Swan', '2010-12-02'),
 ('The Hangover', '2009-06-05'),
 ('The Expendables', '2010-08-03'),
 ('Kick-Ass', '2010-03-22'),
 ('The Godfather', '1972-03-15'),
 ('Grown Ups', '2010-06-24'),
 ('Shutter Island', '2010-02-18'),
 ('Saving Private Ryan', '1998-07-24'),
 ('Despicable Me', '2010-07-08'),
 ('Harry Potter and the Deathly Hallows: Part 1', '2010-10-17'),
 ('Tangled', '2010-11-24'),
 ('TRON: Legacy', '2010-12-10'),
 ('Iron Man 2', '2010-04-28'),
 ('Sherlock Holmes', '2009-01-07'),
 ('Zombieland', '2009-10-07'),
 ('District 9', '2009-08-05'),
 ('Student Services', '2010-01-18'),
 ('RED', '2010-10-13'),
 ('The Godfather: Part II', '1974-12-20'),
 ('Toy Story 3', '2010-06-16'),
 ('How to Train Your Dragon', '2010-03-05'),
 ('Toy Story', '1995-10-30'),
 ('Alice in Wonderland', '2010-03-03'),
 ('Step Up 3D', '2010-08-04'),
 ('The A-Team', '2010-06-10'),
 ('The Social Network', '2010-09

Get top subreddits for all of the corresponding movies

In [3]:
r = praw.Reddit(user_agent='reddit_hype')

with open( "pickled/saved_queries.p", "rb" ) as f:
	subreddits = pickle.load(f)

print(subreddits)



Counter({Subreddit(subreddit_name='deadpool'): 5, Subreddit(subreddit_name='Marvel'): 2, Subreddit(subreddit_name='chimichangas'): 1, Subreddit(subreddit_name='movies'): 1, Subreddit(subreddit_name='Showerthoughts'): 1})
